# Identify cholesterol flip-flop events

A flip-flop event occurs when a molecule - typically a sterol - moves from one leaflet of a bilayer into the opposing leaflet.

We will identify flip-flop events in a ternary mixture of DPPC, DOPC, and Cholesterol simulated by [Smith et al.](https://www.biorxiv.org/content/10.1101/2021.05.24.445501v3).


In [1]:
import pickle

import numpy as np
import MDAnalysis as mda

from lipyphilic.lib.flip_flop import FlipFlop


## Load the topology and trajectory using MDAnalysis

In [2]:
u = mda.Universe("../datafiles/dppc-dopc-chol.tpr", "../datafiles/dppc-dopc-chol.xtc")

## Load the leaflet information

To identify flip-flop events, we first need to know which leaflet each lipid is in at each frame. We will use the results from the notebook on [assigning lipids to leaflets](2-AssignLeaflets.ipynb).

In [3]:
with open("../results/leaflets.pkl", 'rb') as f:
    leaflets = pickle.load(f)
    

## Find flip-flop events

The class `lipyphilic.lib.flip_flop.FlipFlop` finds the frames at which a flip-flop event begins and ends, as well as the direction of travel (upper-to-lower or lower-to-upper).



In [4]:
flip_flop = FlipFlop(
    universe=u,
    lipid_sel="resname CHOL and name ROH",                           # only find flip-flop events for cholesterol
    leaflets=leaflets.filter_leaflets("resname CHOL and name ROH")   # pass only leaflet information on cholesterol
)

Select the frames to use in the analysis (`None` will use every frame):

In [5]:
flip_flop.run(
    start=None,
    stop=None,
    step=None
)


  0%|          | 0/4000 [00:00<?, ?it/s]

## Access the results

The results are then available in the `flipflop.flip_flops` attribute as a NumPy array.

Each row corresponds to an individual flip-flop event. In each row, the four columns correspond to the:
- molecule resindex
- flip-flop start frame
- flip-flop end frame
- the leaflet in which the molecule resides after the flip-flop. 1 and -1 correspond to the upper and lower leaflets respectively.


In [6]:
# We see there were 7743 flip-flop events in total
flip_flop.flip_flops.shape

(7743, 4)

Let's look at the first flip-flop event:

In [7]:
first_event = flip_flop.flip_flops[0]

In [8]:
first_event

array([ 1, 39, 40,  1])

In the above case, cholesterol with residue index 1 left it's original leaflet at frame 39, and entered its new leaflet at frame 40. This new leaflet is the upper leaflet (as the fourth column is equal to 1).

With this information, it is possible to, for example, determine the local lipid environemnt immediately before and after the flip-flop event occured. This is useful to know as [Gu et al.](https://pubs.acs.org/doi/10.1021/acs.jctc.8b00933) showed that translocation is highly influenced by the local lipid environment of cholesterol.

# Specify minimum residence time for successful flip-flops

Due to thermal fluctuations, cholesterol may briefly go to the midplane before returning to its original lealfet. It may also briefly leave the midplane before returning to the hydrophobic core. As such, it is useful to be able to ignore these small fluctuations.

With `FlipFlop`, we can specify the minumum number of frames a molecule must reside in its new leaflet for the flip-flop to be considered successful. We do this using the `frame_cutoff` keyword:

In [9]:
flip_flop = FlipFlop(
    universe=u,
    lipid_sel="name ROH",
    leaflets=leaflets.filter_leaflets("name ROH"),
    frame_cutoff=2,
)

With *frame_cutoff=2*, a molecule must remain in its new leaflet for at least 2 consecutive frames for the flip-flop to be considered successful. If this condition is not, the flip-flop event is ignored.

In [10]:
flip_flop.run(
    start=None,
    stop=None,
    step=None
)


  0%|          | 0/4000 [00:00<?, ?it/s]

Because we have ignored any flip-flop events where cholesterol residues in its new leaflet for one frame only, we see fewer flip-flop events in total:

In [11]:
# We see there are now 4410 flip-flop events in total (compared to 7743 previously)
flip_flop.flip_flops.shape

(4410, 4)

# Calculate the flip-flop rate

`FlipFlop` returns information on whether each event was successful (the molecule resides in the opposing leaflet for at least a given length of time), or not (the molecule went to the midplane but returned to its original leaflet).

This information is stored as a NumPy array in the `flip_flop.flip_flop_success` attribute:

In [12]:
flip_flop.flip_flop_success

array(['Success', 'Success', 'Success', ..., 'Success', 'Success',
       'Success'], dtype='<U7')

There is one value ("Success" or "Fail") for each flip-flop event:

In [13]:
flip_flop.flip_flop_success.size

4410

The number of successful flip-flops can be used to calculate the rate of cholesterol translocation. The flip-flop rate, $k$, is given by the number of successful flip-flop events, $N_{\rm Success}$, divided by the product of the number of cholesterol molecles, ($N_{\rm Chol}$), and the total simulation time, $t_{\rm Seconds}$, used in the analysis:

$$
k = \frac{N_{\rm Success}}{N_{\rm Chol} t_{\rm Seconds}}
$$

In [14]:
cholesterol = u.select_atoms("resname CHOL")
n_cholesterol = cholesterol.n_residues

In [15]:
ps_to_seconds = 1e-12  # convert ps to seconds
total_time = u.trajectory.dt * flip_flop.n_frames * ps_to_seconds

In [16]:
number_successful = np.sum(flip_flop.flip_flop_success == "Success")

In [17]:
flip_flop_rate = number_successful / (n_cholesterol * total_time)  # per second

In [18]:
flip_flop_rate

4252941.176470588

Note
----

See [Baral et al.](https://www.sciencedirect.com/science/article/pii/S0009308420300980) for further discussion on flip-flop in lipid bilayers, including the affect on the flip-flop rate of the buffer size used to assign molecules to the midplane of the bilayer.
